TEST 1

In [477]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer, PorterStemmer

import pickle


In [478]:
data = pd.read_csv('Student Mental Wellness Survey (Responses) - Form responses 1.csv')
data.columns = [f'X{i}' for i in range(1, len(data.columns) + 1)]
data = data.iloc[:, 1:]
print(data.head())

   X2         X3      X4                   X5                X6         X7  \
0  20     Odisha    Male  Bachelor's 3rd Year  Computer Science   Everyday   
1  69     Liquid    Male  Bachelor's 3rd Year  Computer Science   Everyday   
2  20  Hariyana     Male  Bachelor's 3rd Year               BBA  Sometimes   
3  21        UP     Male  Bachelor's 4th Year               CSE         No   
4  56      Delhi  Female  Bachelor's 2nd Year                BA     Rarely   

         X8                  X9       X10       X11        X12       X13  \
0     Often         Weight gain     Often  Everyday   Everyday  Everyday   
1  Everyday           No change  Everyday  Everyday  Sometimes  Everyday   
2    Rarely           No change        No        No         No        No   
3        No           No change        No        No         No        No   
4    Rarely  Increased appetite    Rarely    Rarely     Rarely    Rarely   

        X14             X15      X16  X17  X18  
0    Rarely       No chan

In [479]:
data['X6'] = data['X6'].str.strip().str.lower()
study = data['X6'].value_counts().to_frame()


data['X6'].unique()

array(['computer science', 'bba', 'cse', 'ba', 'senior high school',
       'b.tech', 'btech', 'engeneering', 'btech-cse', 'b.tech.',
       'teacher', 'b.tech computer science', 'b.tech,cse', 'b tech',
       'kiit bhubaneswar', 'btech - cse',
       'react native, machine learning', 'computer science engineering',
       'kiit', 'btech in cse', 'b. tech.', "bachelor's in design",
       'school student', 'bfa', 'political science honours', 'b tech cse',
       'b-tech', 'b. tech',
       'international relations with defence and strategic studies research',
       'b. tech cse', 'biotechnology', 'computer science and engineering',
       'ba llb', 'animation and interior design',
       'b tech, electrical engineering', 'bcom', 'chartered accountancy',
       'mechanical engineering',
       'b.tech. in computer science and engineering',
       'information technology', 'engineering',
       'b.com hons (marketing) with liberal arts', 'bams',
       'b.tech. in artificial intelligenc

In [480]:
variation_mapping = {
    'btech cse': [
        'btech cse', 
        'b tech in computer science', 
        'b.tech cse', 
        'b-tech cse', 
        'b.tech cs (iot - cybersecurity)', 
        'b.tech,cse', 
        'b. tech cse', 
        'computer science and engineering', 
        'b.tech. in computer science and engineering', 
        'b.tech. in artificial intelligence and data science', 
        'btech in computer science engineering', 
        'computer science and communication engineering', 
        'bachelors of technology in computer science', 
        'computer science',
        'computer science engineering',  # Added a comma after 'computer science'
        'bsc computer',  # Added missing comma
        'btech in cse', 
        'react native, machine learning',  
        'btech - cse', 
        'btech(cse)', 
        'btech cse', 
        'btech - cse', 
        'btech in cse', 
        'computer science', 
        'b.tech computer science', 
        'computer science engineering', 
        'information technology', 
        'kiit bhubaneswar', 
        'kiit', 
        'btech(cse)'
    ],
    'ba': [
        'ba', 
        'ba hons. - psychology', 
        'ba psychology honors'
    ],
    'bbs bcom mba bba': [
        'bbs', 
        'bcom honors',
        'bcom', 
        'mba', 
        'b.com hons (marketing) with liberal arts', 
        'b.com hons.', 
        'b com', 
        'b.com hons.',  # Removed duplicate
        'mba - operations and scm', 
        'bcom accounting and finance','bba','mba',
    ],
    'engineering': [
        'engineering', 
        'engineering',  # Removed duplicated 'engineering'
        'engineer',  # Corrected typo from 'enginer' to 'engineer'
        'btech aerospace engineering', 
        'b.tech aerospace engineering', 
        'aerospace engineering','b.tech', 
        'b.tech.', 
        'b tech', 
        'b. tech.', 
        'b tech cse', 
        'b-tech', 
        'iitbombay', 
        'b. tech',
        'biotechnology'  
        'b tech, electrical engineering', 
        'mechanical engineering'
    ],
    'bsc': [
        'bsc computer science', 
        'bsc. applied psychology', 
        'bsc', 
        'bsc it', 
        'bsc nursing',  # Removed duplicate
        'bsc. applied psychology', 
        'bsc', 'bdes', 
        'bachelors in design (animation, vfx and illustration)',
        'bsc it','mathematics and computing',"bachelor's in design", 
        "bachelor's of design",  
        "bachelor's of applied it"  # Corrected typo from "bachelor;s of applied it" to "bachelor's of applied it"
    ],

    'books': ['books'],
    'commerce': ['commerce'],
    'medical': ['bachelors in dental surgery','bachelors in dental surgery', 'bsc nursing'],  # Corrected a missing bracket
    'others': [
         
        'senior high school', 
        'teacher',
        'school student', 
        'bfa', 
        'international relations with defence and strategic studies research', 
        'ba llb', 
        'animation and interior design', 
        'chartered accountancy', 
        'bams', 
        'architecture', 
        'studied architecture', 
        'preparing for civil services !!', 
        'masters in economics', 
        'finance in mba', 
        'no', 
         
        'design', 
        '-', 
        'neet', 
        'fashion design', 
        'b.des', 
        'math', 
         
        'film studies', 
        
        'in job', 
        'windchill+', 
        'fashion', 
        'political science honours', 
        
    ]
}


In [481]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import matplotlib.pyplot as plt

# Sample data
df = data['X6'].unique()

# Preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

import re



def preprocess_text(text):
    text = text.lower()  # Convert text to lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespaces
    # Replace variations with their common representation
    for common_term, variations in variation_mapping.items():
        for variation in variations:
            text = text.replace(variation, common_term)
    return text

processed_data = [preprocess_text(text) for text in df]



# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_data)

# Clustering
num_clusters = 10
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(tfidf_matrix)
cluster_labels = kmeans.labels_

# Dimensionality reduction for visualization
pca = PCA(n_components=2)
tfidf_matrix_reduced = pca.fit_transform(tfidf_matrix.toarray())

# Post-processing
cluster_centers = kmeans.cluster_centers_

for i in range(num_clusters):
    cluster_data = np.array(df)[cluster_labels == i]
    print(f"Cluster {i}:")
    print(cluster_data)
    print("----------")


Cluster 0:
["bachelor's in design" "bachelor's of design" 'bsc. applied psychology'
 'bdes' 'bsc' 'mathematics and computing' 'bsc it' 'bsc nursing']
----------
Cluster 1:
['senior high school' 'teacher' 'school student' 'bfa'
 'international relations with defence and strategic studies research'
 'ba llb' 'animation and interior design' 'chartered accountancy' 'bams'
 'graphic design' 'architecture' 'studied architecture'
 'preparing for civil services !!' 'masters in economics' 'no' 'design'
 '-' 'neet' 'fashion design' 'b.des' 'math' 'film studies' 'in job'
 'windchill+' 'fashion']
----------
Cluster 2:
['computer science' 'cse' 'btech' 'b.tech computer science' 'b.tech,cse'
 'kiit bhubaneswar' 'btech - cse' 'react native, machine learning' 'kiit'
 'btech in cse' 'b. tech cse' 'computer science and engineering'
 'b.tech. in computer science and engineering' 'information technology'
 'b.tech. in artificial intelligence and data science'
 'b.tech cs (iot - cybersecurity)' 'btech in co

In [482]:
data['X3'] = data['X3'].str.strip().str.lower().replace({
    'uttar pradesh': 'Uttar Pradesh',
    'up': 'Uttar Pradesh',
    'madhya pradesh': 'Madhya Pradesh',
    'mp': 'Madhya Pradesh',
    'maharashtra': 'Maharashtra',
    'maharastra': 'Maharashtra',
    'mahrashtra': 'Maharashtra',
    'haryana': 'Haryana',
    'odisha': 'Odisha',
    'west bengal': 'West Bengal',
    'chhattisgarh': 'Chhattisgarh',
    'karnataka': 'Karnataka',
    'delhi': 'Delhi',
    'bhubaneswar': 'Odisha',
    'faridabad': 'Haryana',
    'delhi': 'Delhi',
    'rajasthan': 'Rajasthan',
    'rajisthan': 'Rajasthan',
    'tamil nadu': 'Tamil Nadu',  
    'Karnataka': 'Karnataka', 
    'delhi ncr': 'Delhi',
})

# Get the unique values in the cleaned Location column
unique_locations = data['X3'].unique()

print(unique_locations)


['Odisha' 'liquid' 'hariyana' 'Uttar Pradesh' 'Delhi' 'Rajasthan'
 'Haryana' 'bihar' 'Maharashtra' 'West Bengal' '-' 'Madhya Pradesh'
 'jharkhand' 'Tamil Nadu' 'Chhattisgarh' 'Karnataka' 'not from india'
 'gujrat' 'uttarpradesh' 'assam' 'goa' 'punjab']


These columns only take 2-5 values hence one hot encoding is done

In [483]:
columns_to_encode = ['X4','X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15']
data= pd.get_dummies(data, columns=columns_to_encode)
print(data.head())

   X2             X3                   X5                X6      X16  X17  \
0  20         Odisha  Bachelor's 3rd Year  computer science  Nothing  Yes   
1  69         liquid  Bachelor's 3rd Year  computer science     nope  Yes   
2  20       hariyana  Bachelor's 3rd Year               bba       No   No   
3  21  Uttar Pradesh  Bachelor's 4th Year               cse      NaN  Yes   
4  56          Delhi  Bachelor's 2nd Year                ba      NaN   No   

   X18  X4_Female  X4_Male  X4_Others  ...  X13_Rarely  X13_Sometimes  \
0   No      False     True      False  ...       False          False   
1  Yes      False     True      False  ...       False          False   
2  Yes      False     True      False  ...       False          False   
3  Yes      False     True      False  ...       False          False   
4   No       True    False      False  ...        True          False   

   X14_Everyday  X14_No  X14_Often  X14_Rarely  X14_Sometimes  X15_No change  \
0         False   

These columns can take more than one value so only the top 10 are taken

In [484]:
top_categories = 10
columns_to_encode = ['X3', 'X5', 'X6']
encoded_columns = pd.DataFrame()
for column in columns_to_encode:
    top_categories_series = data[column].value_counts().nlargest(top_categories)
    top_categories_list = top_categories_series.index.tolist()
    
    encoded = pd.get_dummies(data[column].apply(lambda x: x if x in top_categories_list else 'Other'), prefix=column)
    encoded_columns = pd.concat([encoded_columns, encoded], axis=1)
data_encoded = pd.concat([data.drop(columns=columns_to_encode), encoded_columns], axis=1)
print(data_encoded.head())

   X2      X16  X17  X18  X4_Female  X4_Male  X4_Others  X4_Prefer not to say  \
0  20  Nothing  Yes   No      False     True      False                 False   
1  69     nope  Yes  Yes      False     True      False                 False   
2  20       No   No  Yes      False     True      False                 False   
3  21      NaN  Yes  Yes      False     True      False                 False   
4  56      NaN   No   No       True    False      False                 False   

   X7_Everyday  X7_No  ...  X6_b.tech  X6_btech  X6_btech cse  \
0         True  False  ...      False     False         False   
1         True  False  ...      False     False         False   
2        False  False  ...      False     False         False   
3        False   True  ...      False     False         False   
4        False  False  ...      False     False         False   

   X6_btech in cse  X6_commerce  X6_computer science  \
0            False        False                 True   
1         

In [485]:
sia = SentimentIntensityAnalyzer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()


data_encoded['X16'] = data_encoded['X16'].astype(str)
negative_responses = ['no', 'nope', 'nothing', 'na','nope.','nah','not at all','no.', ]


data_encoded['X16_sentiment'] = data_encoded['X16'].apply(lambda x: 0.0 if any(token in [stemmer.stem(lemmatizer.lemmatize(word)) for word in nltk.word_tokenize(x.lower())] for token in negative_responses) else sia.polarity_scores(x)['compound'] if isinstance(x, str) else 0.0)

data_encoded.head()
def merge_columns(row):
    if row['X18'] == 'Yes':
        return True
    elif row['X17'] == 'Yes':
        return True
    else: 
        return False

# Apply the function to create the new column
data_encoded['Output'] = data_encoded.apply(merge_columns, axis=1)
data_encoded.drop(['X18', 'X17', 'X16'], axis=1, inplace=True)
data_encoded

,X2,X4_Female,X4_Male,X4_Others,X4_Prefer not to say,X7_Everyday,X7_No,X7_Often,X7_Rarely,X7_Sometimes,...,X6_btech cse,X6_btech in cse,X6_commerce,X6_computer science,X6_computer science and engineering,X6_computer science engineering,X6_cse,X6_engineering,X16_sentiment,Output
0,20,False,True,False,False,True,False,False,False,False,...,False,False,False,True,False,False,False,False,0.0000,True
1,69,False,True,False,False,True,False,False,False,False,...,False,False,False,True,False,False,False,False,0.0000,True
2,20,False,True,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,0.0000,True
3,21,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,True,False,0.0000,True
4,56,True,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,0.0000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,20,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,0.0000,False
178,20,True,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,0.0000,False
179,19,True,False,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,-0.3412,False
180,18,True,False,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,0.0000,True


In [486]:
data_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 182 entries, 0 to 181
Data columns (total 83 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   X2                                   182 non-null    int64  
 1   X4_Female                            182 non-null    bool   
 2   X4_Male                              182 non-null    bool   
 3   X4_Others                            182 non-null    bool   
 4   X4_Prefer not to say                 182 non-null    bool   
 5   X7_Everyday                          182 non-null    bool   
 6   X7_No                                182 non-null    bool   
 7   X7_Often                             182 non-null    bool   
 8   X7_Rarely                            182 non-null    bool   
 9   X7_Sometimes                         182 non-null    bool   
 10  X8_Everyday                          182 non-null    bool   
 11  X8_No                           

In [487]:

data_encoded.to_csv('encoded_data.csv', index=False)
